In [1]:
import pandas as pd
import shutil, os
import numpy as np

In [2]:
df = pd.read_csv('r.csv')
df.head()

,parcel_name,sws_rs,sws_gb,sws_ms,sws_fa,sws_rw,sws_total
0,gmd_17Networks_17,0.166971,0.051392,0.107496,-0.069671,0.184833,0.111557
1,gmd_17Networks_16,0.118898,0.035596,0.026529,0.069384,0.099563,0.092149
2,gmd_17Networks_15,0.040398,0.026939,0.022451,0.109657,0.005116,0.059787
3,gmd_17Networks_14,-0.044211,-0.076722,-0.072343,0.049435,-0.054085,-0.040322
4,gmd_17Networks_13,0.116004,-0.003638,0.144275,-0.098356,0.146914,0.085295


In [3]:
df1 = df[['parcel_name' ,'sws_gb']]
df1.head()

,parcel_name,sws_gb
0,gmd_17Networks_17,0.051392
1,gmd_17Networks_16,0.035596
2,gmd_17Networks_15,0.026939
3,gmd_17Networks_14,-0.076722
4,gmd_17Networks_13,-0.003638


In [4]:
df1.shape

(2718, 2)

In [5]:
df2 = df1.dropna()

In [6]:
df2.shape

(2718, 2)

In [7]:
df2.insert(1, 'parc_name', np.nan)

In [8]:
df2.head()

,parcel_name,parc_name,sws_gb
0,gmd_17Networks_17,NaN,0.051392
1,gmd_17Networks_16,NaN,0.035596
2,gmd_17Networks_15,NaN,0.026939
3,gmd_17Networks_14,NaN,-0.076722
4,gmd_17Networks_13,NaN,-0.003638


In [9]:
df2 = df2.reset_index()

In [10]:
df2.head()

,index,parcel_name,parc_name,sws_gb
0,0,gmd_17Networks_17,NaN,0.051392
1,1,gmd_17Networks_16,NaN,0.035596
2,2,gmd_17Networks_15,NaN,0.026939
3,3,gmd_17Networks_14,NaN,-0.076722
4,4,gmd_17Networks_13,NaN,-0.003638


In [14]:
df2 = df.drop('index', axis = 1)

In [15]:
df = df2

In [16]:
df.head()

,parcel_name,parc_name,sws_gb
0,gmd_17Networks_17,NaN,0.051392
1,gmd_17Networks_16,NaN,0.035596
2,gmd_17Networks_15,NaN,0.026939
3,gmd_17Networks_14,NaN,-0.076722
4,gmd_17Networks_13,NaN,-0.003638


In [17]:
markers = ['gmd_', 'zALFF_', 'zfALFF_', 'zReHo_', 'zDCwei_', 'zDCbin_']
for i,parc in enumerate(df["parcel_name"]):
    #for i in range(df.shape[0]):
    for marker in markers:
        if parc.find(marker)==0:
            df["parc_name"][i] = df["parcel_name"][i][len(marker):]
            #print(df["sws_total"][i])
            col = marker + "sws_gb_r" 
            if col not in df:
                df.insert(2, col, np.nan)
            df.iloc[i, df.columns.get_loc(col)] = df["sws_gb"][i]

<ipython-input-17-3dcbac682cc0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["parc_name"][i] = df["parcel_name"][i][len(marker):]
/home/tara/Programming/env/lib/python3.8/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [18]:
df.shape

(2718, 9)

In [19]:
#df.to_csv('r_reformed.csv')

### merging duplicates

In [20]:
#df = pd.read_csv('r_reformed.csv')

In [21]:
df.shape

(2718, 9)

In [22]:
df2 = df.copy()
inds = df2['parc_name'].drop_duplicates().reset_index(drop=True)
df2.set_index('parc_name', inplace=True)
df3 = pd.DataFrame(columns = df2.columns)
for ind in inds:
    region = df2.loc[[ind]]
    if region.shape[0] > 1 :
        reg = region
        cols = reg.columns
        for col in cols:
            reg[col] = reg[col].sum()
        reg = reg.drop_duplicates()
        df3 = pd.concat([df3, reg])
        continue
    df3 = pd.concat([df3, region])

In [23]:
df3.shape

(453, 8)

In [24]:
df3.to_csv('SWSgb_r_reformed.csv')